In [14]:
# Save the books

import requests

# Get a request and save it as a txt file

books = {
    "863": "The Mysterious Affair at Styles",
    "1155": "The Secret Adversary",
    "58866": "The Murder on the Links",
    "61168": "The Man in the Brown Suit",
    "61262": "Poirot Investigates",
    "65238": "The Secret of Chimneys",
    "66446": "The Plymouth Express Affair",
    "67160": "The Hunter's Lodge Case",
    "67173": "The Missing Will",
    "69087": "The Murder of Roger Ackroyd",
    "70114": "The Big Four",
    "72824": "The Mystery of the Blue Train"
}

# Go through the dict

for id, title in books.items():
    res = requests.get(f'http://www.gutenberg.org/ebooks/{id}.txt.utf-8')
    # Write to txt file

    cleaned_text = res.text.encode('utf-8', errors='replace').decode('utf-8')
    
    filename = title.replace(" ", "-") + ".txt"
    # with open(filename, 'w', encoding='utf-8') as file:
    #     # Clean res.text to have nothing that cannot be encoded in a .write() call
    #     file.write(cleaned_text)
    #     pass

In [15]:
# Python program to generate word vectors using Word2Vec

# importing all necessary modules
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action='ignore')


# Reads file
sample = open("books/Poirot-Investigates.txt", encoding="utf-8")
s = sample.read()

# Replaces escape character with space
f = s.replace("\n", " ")

data = []

# iterate through each sentence in the file
for i in sent_tokenize(f):
	temp = []

	# tokenize the sentence into words
	for j in word_tokenize(i):
		temp.append(j.lower())

	data.append(temp)

# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count=1,
								vector_size=100, window=5)

# Print results
print("Cosine similarity between 'murder' " +
	"and 'investigate' - CBOW : ",
	model1.wv.similarity('murder', 'investigate'))

print("Cosine similarity between 'murder' " +
	"and 'said' - CBOW : ",
	model1.wv.similarity('murder', 'said'))

# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count=1, vector_size=100,
								window=5, sg=1)

# Print results
print("Cosine similarity between 'murder' " +
	"and 'investigate' - Skip Gram : ",
	model2.wv.similarity('murder', 'investigate'))

print("Cosine similarity between 'murder' " +
	"and 'said' - Skip Gram : ",
	model2.wv.similarity('murder', 'said'))


Cosine similarity between 'murder' and 'investigate' - CBOW :  0.98677754
Cosine similarity between 'murder' and 'said' - CBOW :  0.98697877
Cosine similarity between 'murder' and 'investigate' - Skip Gram :  0.9960083
Cosine similarity between 'murder' and 'said' - Skip Gram :  0.8106532
